In [2]:
import sddk
import pandas as pd
import tabulate
pd.options.display.max_columns = 1000 # to see all columns
import json
import re
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import pickle

import nltk
from nltk.util import bigrams

In [3]:
LIREg = sddk.read_file("LIREg.geojson", "gdf", "a9237c5ea642d4714bcdefb03f70a1f4")

reading file located in a public folder


In [5]:
EDH_overlap_all = LIREg[(LIREg["EDH-ID"].notnull()) & (LIREg["EDCS-ID"].notnull())] 
EDH_overlap = EDH_overlap_all[~EDH_overlap_all["type_of_inscription_clean"].str.contains("NULL")]

Now we can focus on some EDCS attributes (i.e. firt 28 columns) which might be good predictors of `type_of_inscription_clean` in EDH. First, look at `status_list`:

# Applying the model

In [6]:
# load the model and vectorizer back
classifier = pickle.load(open('../data/large_data/et_model_v1.0.sav', 'rb'))
vectorizer = pickle.load(open('../data/large_data/et_vectorizer_v1.0.sav','rb'))

In [7]:
# we will apply the model to inscriptions which are only in EDCS
EDCS_unique = LIREg[(LIREg["EDH-ID"].isnull()) & (LIREg["EDCS-ID"].notnull())] 
EDCS_unique.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry
53823,EDCS-71300269,"AE 2014, 01203",Achaia,Achaia,Mavrommati / Mavromati / Messini / Messene,"['Mavrommati', 'Mavromati', 'Messini', 'Messene']",96,,"['sigilla impressa', 'tituli fabricationis', '...",tituli fabricationis,viri,sigilla impressa,,,De Rasticanis,De Rasticanis,None,69.0,96,sigilla impressa; tituli fabricationis; viri,37.179412,21.924179,None,opus figlinae,None,None,None,De Rasticanis,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,96.0,Latin,69.0,"['21.9241791', '37.1794122']",De Rasticanis,big,False,Messene,59,22337.0,"[22.008029, 37.051363]",middle,0.153060,owner/artist inscription,0.785702,POINT (21.92418 37.17941)
53824,EDCS-21200140,"CIL 03, 00079 = CIL 03, 14147,6 = IDakke 00028",Aegyptus,Aegyptus,Ad-Dakka / El-Dakka / Dakke / Pselqet / Pselki...,"['Ad-Dakka', 'El-Dakka', 'Dakke', 'Pselqet', '...",109,,"['milites', 'tituli fabricationis', 'tituli sa...","['tituli fabricationis', 'tituli sacri']","['milites', 'tria nomina', 'viri']",{},,,Deo Magno Mercurio / adoravit vexillu / leg(io...,Deo Magno Mercurio / adoravit vexillu / leg(io...,http://db.edcs.eu/epigr/partner.php?s_language...,109.0,109,milites; tituli fabricationis; tituli sacri; t...,23.200000,32.750000,None,None,None,None,None,Deo Magno Mercurio adoravit vexillu legionis I...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,109.0,Latin,109.0,"['32.750000', '23.200000']",Deo Magno Mercurio adoravit vexillu legionis I...,rest,False,Elephantine,126,33806.0,"[32.933333, 24.083333]",big,0.902158,owner/artist inscription,0.850080,POINT (32.75000 23.20000)
53825,EDCS-21200142,"CIL 03, 00081 = CIL 03, 13584 = IDakke 00064a",Aegyptus,Aegyptus,Ad-Dakka / El-Dakka / Dakke / Pselqet / Pselki...,"['Ad-Dakka', 'El-Dakka

In [10]:
# first create bigrams
def get_bigrams_underscore(inscr_text):
    try:
        inscr_bigrams = [" ".join(el) for el in list(bigrams(inscr_text.split()))]
        inscr_bigrams_ = [bigram.replace(" ", "_") for bigram in inscr_bigrams]
    except:
        inscr_bigrams_ = []
    return inscr_bigrams_

EDCS_unique["bigrams"] = EDCS_unique["clean_text_interpretive_word"].apply(get_bigrams_underscore)

In [11]:
def combine_status_list_and_bigrams(el_status, el_bigrams, el_material):
    # preprocess status:
    try: el_status = eval(el_status)
    except: pass
    if isinstance(el_status, list):
        new_el_status = el_status
    elif isinstance(el_status, str):
        new_el_status = [el_status]
    else: 
        new_el_status = []
    new_el_status = [el.replace(" ", "_") for el in new_el_status]
    # preprocess material:
    if el_material is None:
        el_material = []    
    else:
        el_material = el_material.partition(": ")[0] # 
        el_material = el_material.split(", ")
        el_material = [el.replace(" ", "_").replace("?", "") for el in el_material]
    # combine status, bigrams and material
    new_el = new_el_status + el_bigrams + el_material
    new_el = " ".join(new_el)
    return new_el, new_el_status, el_material

In [12]:
# we need to extract all predictors from the dataframe
# into `status_bigrams_list_apply` (list of lists)
status_bigrams_list_apply = []
status_list = []
material_list = []
for el_status, el_bigrams, el_material in zip(EDCS_unique["status_list"].tolist(), EDCS_unique["bigrams"].tolist(), EDCS_unique["Material"].tolist()):
    new_el, new_status, el_material = combine_status_list_and_bigrams(el_status, el_bigrams, el_material)
    status_bigrams_list_apply.append(new_el)
    status_list.append(new_status)
    material_list.extend(el_material)

In [13]:
X_applied = vectorizer.transform(status_bigrams_list_apply)

In [15]:
prediction = classifier.predict(X_applied)
prediction[:10]

array(['owner/artist inscription', 'owner/artist inscription', 'epitaph',
       'votive inscription', 'votive inscription', 'epitaph', 'epitaph',
       'honorific inscription', 'honorific inscription',
       'honorific inscription'], dtype='<U31')

In [16]:
classifier.classes_

array(['acclamation', 'adnuntiatio', 'assignation inscription',
       'boundary inscription', 'building/dedicatory inscription',
       'calendar', 'defixio', 'elogium', 'epitaph',
       'honorific inscription', 'identification inscription', 'label',
       'letter', 'list', 'mile-/leaguestone', 'military diploma',
       'owner/artist inscription', 'prayer', 'private legal inscription',
       'public legal inscription', 'seat inscription',
       'votive inscription'], dtype='<U31')

In [27]:
probs = [np.max(pred) for pred in classifier.predict_proba(X_applied)]
probs[:10]

[1.0,
 0.41,
 0.4293233082706769,
 0.49763033175355514,
 0.64,
 0.97,
 1.0,
 0.72,
 0.91,
 0.78]

In [17]:
prediction_probs = [(classifier.classes_[np.argmax(pred)], pred[np.argmax(pred)]) for pred in classifier.predict_proba(X_applied)]
prediction_probs[:10]

[('owner/artist inscription', 1.0),
 ('owner/artist inscription', 0.41),
 ('epitaph', 0.4293233082706769),
 ('votive inscription', 0.49763033175355514),
 ('votive inscription', 0.64),
 ('epitaph', 0.97),
 ('epitaph', 1.0),
 ('honorific inscription', 0.72),
 ('honorific inscription', 0.91),
 ('honorific inscription', 0.78)]

In [18]:
len(prediction_probs)

83482

In [20]:
for threshold in [thres / 100 for thres in range(40, 100, 5)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    print("prob. threshold: {0}, proportion of inscr.: {1}".format(threshold, proportion))

prob. threshold: 0.4, proportion of inscr.: 0.9596559737428427
prob. threshold: 0.45, proportion of inscr.: 0.8660789152152559
prob. threshold: 0.5, proportion of inscr.: 0.8500874439999042
prob. threshold: 0.55, proportion of inscr.: 0.8325387508684506
prob. threshold: 0.6, proportion of inscr.: 0.8172540188304066
prob. threshold: 0.65, proportion of inscr.: 0.7999808341918018
prob. threshold: 0.7, proportion of inscr.: 0.7821925684578711
prob. threshold: 0.75, proportion of inscr.: 0.7641168156009679
prob. threshold: 0.8, proportion of inscr.: 0.7404350638460986
prob. threshold: 0.85, proportion of inscr.: 0.7088833520998539
prob. threshold: 0.9, proportion of inscr.: 0.6791763493926835
prob. threshold: 0.95, proportion of inscr.: 0.6157015883663545


In [21]:
threshold_results = []
for threshold in [thres / 100 for thres in range(40, 100, 5)]:
    above_threshold = [tup for tup in prediction_probs if tup[1] >= threshold]
    proportion = len(above_threshold) / len(prediction_probs)
    threshold_results.append({"threshold (=/>)" : threshold, "proportion" : np.round(proportion, 2), "N" : len(above_threshold)})
model_results_df = pd.DataFrame(threshold_results)
model_results_df

,threshold (=/>),proportion,N
0,0.40,0.96,80114
1,0.45,0.87,72302
2,0.50,0.85,70967
3,0.55,0.83,69502
4,0.60,0.82,68226
5,0.65,0.80,66784
6,0.70,0.78,65299
7,0.75,0.76,63790
8,0.80,0.74,61813
9,0.85,0.71,59179


In [25]:
print(model_results_df.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
 threshold (=/>) &  proportion &      N \\
\midrule
            0.40 &        0.96 &  80114 \\
            0.45 &        0.87 &  72302 \\
            0.50 &        0.85 &  70967 \\
            0.55 &        0.83 &  69502 \\
            0.60 &        0.82 &  68226 \\
            0.65 &        0.80 &  66784 \\
            0.70 &        0.78 &  65299 \\
            0.75 &        0.76 &  63790 \\
            0.80 &        0.74 &  61813 \\
            0.85 &        0.71 &  59179 \\
            0.90 &        0.68 &  56699 \\
            0.95 &        0.62 &  51400 \\
\bottomrule
\end{tabular}



In [22]:
model_results_df.to_csv("../data/classifier_EDCS_results.csv")

# Combining the data back

In [28]:
EDCS_unique["type_of_inscription_auto"] = prediction
EDCS_unique["type_of_inscription_auto_prob"] = probs

In [29]:
EDCS_unique.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry,bigrams
53823,EDCS-71300269,"AE 2014, 01203",Achaia,Achaia,Mavrommati / Mavromati / Messini / Messene,"['Mavrommati', 'Mavromati', 'Messini', 'Messene']",96,,"['sigilla impressa', 'tituli fabricationis', '...",tituli fabricationis,viri,sigilla impressa,,,De Rasticanis,De Rasticanis,None,69.0,96,sigilla impressa; tituli fabricationis; viri,37.179412,21.924179,None,opus figlinae,None,None,None,De Rasticanis,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,96.0,Latin,69.0,"['21.9241791', '37.1794122']",De Rasticanis,big,False,Messene,59,22337.0,"[22.008029, 37.051363]",middle,0.153060,owner/artist inscription,1.000000,POINT (21.92418 37.17941),[De_Rasticanis]
53824,EDCS-21200140,"CIL 03, 00079 = CIL 03, 14147,6 = IDakke 00028",Aegyptus,Aegyptus,Ad-Dakka / El-Dakka / Dakke / Pselqet / Pselki...,"['Ad-Dakka', 'El-Dakka', 'Dakke', 'Pselqet', '...",109,,"['milites', 'tituli fabricationis', 'tituli sa...","['tituli fabricationis', 'tituli sacri']","['milites', 'tria nomina', 'viri']",{},,,Deo Magno Mercurio / adoravit vexillu / leg(io...,Deo Magno Mercurio / adoravit vexillu / leg(io...,http://db.edcs.eu/epigr/partner.php?s_language...,109.0,109,milites; tituli fabricationis; tituli sacri; t...,23.200000,32.750000,None,None,None,None,None,Deo Magno Mercurio adoravit vexillu legionis I...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,109.0,Latin,109.0,"['32.750000', '23.200000']",Deo Magno Mercurio adoravit vexillu legionis I...,rest,False,Elephantine,126,33806.0,"[32.933333, 24.083333]",big,0.902158,owner/artist inscription,0.410000,POINT (32.75000 23.20000),"[Deo_Magno, Magno_Mercurio, Mercurio_adoravit,..."
53825,EDCS-21200142,"CIL 03, 00081 = CIL 03, 13584 = IDakke 00064a",Aegyptus,Aegypt

In [30]:
# prepare last part of the data - inscriptions we have only in EDH
# in this case, we can also use the information we already have
EDH_unique = LIREg[(LIREg["EDH-ID"].notnull()) & (LIREg["EDCS-ID"].isnull())] 
EDH_unique["type_of_inscription_auto"] = EDH_unique["type_of_inscription_clean"]
EDH_unique["type_of_inscription_auto_prob"] = [1] * len(EDH_unique)

In [31]:
EDH_overlap_all = LIREg[(LIREg["EDH-ID"].notnull()) & (LIREg["EDCS-ID"].notnull())] 
EDH_overlap_all["type_of_inscription_auto"] = EDH_overlap_all["type_of_inscription_clean"]
EDH_overlap_all["type_of_inscription_auto_prob"] = [1] * len(EDH_overlap_all)

In [32]:
print("the dataset we used for training and testing (+NULLs): ", EDH_overlap_all.shape)
print("inscriptions in EDH only: ", EDH_unique.shape)
print("inscriptions in EDCS only, we wanted to classify: ", EDCS_unique.shape)

the dataset we used for training and testing (+NULLs):  (49916, 112)
inscriptions in EDH only:  (3907, 112)
inscriptions in EDCS only, we wanted to classify:  (83482, 113)


In [33]:
# combine together
LIRE_autotypes = pd.concat([EDH_overlap_all, EDH_unique, EDCS_unique])
LIRE_autotypes.head()

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry,bigrams
0,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{},{},121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,honorific inscription,1.0,POINT (20.76680 39.45120),NaN
1,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{},,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,276,Augusti/Augustae; miliaria; viri,37.45450

In [34]:
LIRE_autotypes.drop(["bigrams"], axis=1, inplace=True)

In [35]:
LIRE_autotypes.shape

(137305, 112)

In [37]:
LIRE_autotypes.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry
0,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{},{},121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,honorific inscription,1.0,POINT (20.76680 39.45120)
1,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{},,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,276,Augusti/Augustae; miliaria; viri,37.454501,22.420877,

In [38]:
len(LIRE_autotypes[LIRE_autotypes["type_of_inscription_auto_prob"]>=0.6])

122049

In [39]:
len(LIRE_autotypes[LIRE_autotypes["type_of_inscription_auto"]=="NULL"])

4339

In [40]:
def apply_threshold(inscr_type, prob):
    if (prob < 0.6) or (inscr_type == "NULL"):
        return None 
    else:
        return inscr_type
LIRE_autotypes["type_of_inscription_auto"] =  LIRE_autotypes.apply(lambda row: apply_threshold(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [41]:
def prob_to_zero(inscr_type, prob):
    if inscr_type == None:
        return 0
    else:
        return prob
LIRE_autotypes["type_of_inscription_auto_prob"] =  LIRE_autotypes.apply(lambda row: prob_to_zero(row["type_of_inscription_auto"], row["type_of_inscription_auto_prob"]), axis=1)

In [42]:
len(LIRE_autotypes[LIRE_autotypes["type_of_inscription_auto"].notnull()])

117710

In [43]:
len(LIRE_autotypes[(LIRE_autotypes["type_of_inscription_auto"].notnull()) & (LIRE_autotypes["type_of_inscription_auto_prob"] > 0.55)])

117710

In [47]:
117710 / 137305

0.8572885182622628

In [44]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root/SDAM_data/LIRE", "648597@au.dk")

Your ScienceData username (e.g. '123456@au.dk'): 648597@au.dk
Your ScienceData password: ········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/SDAM_data/LIRE/


In [45]:
LIRE_autotypes.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry
0,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{},{},121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,honorific inscription,1.0,POINT (20.76680 39.45120)
1,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{},,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,276,Augusti/Augustae; miliaria; viri,37.454501,22.420877,

In [46]:
s.write_file("LIREg.geojson", LIRE_autotypes)

A file with the same name ("LIREg.geojson") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'geopandas.geodataframe.GeoDataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/LIRE/LIREg.geojson"
